In [1]:
%%capture
!pip install gradio
!pip install accelerate -U
!pip install transformers huggingface_hub
!pip install gliner[gpu]

In [6]:
from gliner import GLiNER
import torch

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print(device)

model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1").to(device)

cuda


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.77k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

/venv/main/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
label_mapping = {
    "ORG": "organization",
    "PERS": "person",
    "LOC": "location",
    "MON": "monetary value",
    "PCT": "percentage",
    "DATE": "date",
    "TIME": "timestamp",
    "PERIOD": "time period",
    "JOB": "job title",
    "DOC": "document name",
    "QUANT": "quantity",
    "ART": "artifact",
    "MISC": "miscellaneous"
}

labels = list(label_mapping.values())
reverse_label_mapping = {v: k for k, v in label_mapping.items()}

In [8]:
from tqdm import tqdm
import pandas as pd
import json

df = pd.read_csv("test.csv")

results = []
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    _, text, _ = row
    entities = model.predict_entities(text, labels)
    
    formatted_entities = [{
        "label": reverse_label_mapping[ent["label"]], 
        "text": ent["text"],
        "start": ent["start"],
        "end": ent["end"]
    } for ent in entities]
    
    results.append(json.dumps(formatted_entities, ensure_ascii=False))
    
df["pred"] = results

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
100%|██████████| 5087/5087 [00:43<00:00, 116.14it/s]


In [10]:
df.to_csv("zero_shot_gliner_multi-v2.1.csv", index=False)